<a href="https://colab.research.google.com/github/jainstuti/Tweets_classification/blob/main/knn_TMLC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
#importing libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [55]:
#importing dataset
dataset = pd.read_csv('Twitter_Data.csv')

In [56]:
dataset=dataset.head(20000)

In [57]:
dataset.shape

(20000, 2)

In [58]:
# to get the values in categorical data and their counts in the data
dataset['category'].value_counts()

 1.0    8374
 0.0    6807
-1.0    4819
Name: category, dtype: int64

In [59]:
dataset.describe()

,category
count,20000.000000
mean,0.177750
std,0.792519
min,-1.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [60]:
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   clean_text  19999 non-null  object 
 1   category    20000 non-null  float64
dtypes: float64(1), object(1)
memory usage: 312.6+ KB


In [61]:
dataset.isnull().sum()


clean_text    1
category      0
dtype: int64

In [62]:
is_NaN = dataset.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = dataset[row_has_NaN]

print(rows_with_NaN)

    clean_text  category
148        NaN       0.0


In [63]:
dataset=dataset.dropna()

In [64]:
dataset = dataset.reset_index()

In [65]:
dataset.shape

(19999, 3)

In [66]:
dataset.head()

,index,clean_text,category
0,0,when modi promised “minimum government maximum...,-1.0
1,1,talk all the nonsense and continue all the dra...,0.0
2,2,what did just say vote for modi welcome bjp t...,1.0
3,3,asking his supporters prefix chowkidar their n...,1.0
4,4,answer who among these the most powerful world...,1.0


In [67]:
dataset =dataset.drop(['index'], axis = 1)

In [68]:
#cleaning the text
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 19999):
  tweet = re.sub("[^a-zA-Z]", " ", dataset['clean_text'][i])
  tweet = tweet.lower()
  tweet = tweet.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  tweet = [ps.stem(word) for word in tweet if not word in set(all_stopwords)]
  tweet = ' '.join(tweet)
  corpus.append(tweet)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [69]:
#creating bag of words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 3000)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, 1].values

In [70]:
X.shape

(19999, 3000)

In [71]:
#slpiiting data into training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [72]:
#knn
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [73]:
y_pred = classifier.predict(X_test)

In [74]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[  98  835   37]
 [   5 1354   11]
 [  34 1340  286]]


0.4345

In [75]:
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)
print(np.mean(recall))
print(np.mean(precision))

0.42054708411492303
0.6517646626936087


In [76]:
f1Score = 2 * precision * recall / (precision + recall)
print(np.mean(f1Score))

0.33889385207181394
